In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR ctau-BDT-based (trained on [5,50]GeV): run on all signal mass points [5, 100] GeV

In [4]:
ct = 100

## Background

In [5]:
prefix = './coffea/skimmed/'
suffix = f'_SR_BDT-based_ctau-{ct}_trained_5to50'

In [6]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [7]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to50_Background_Efficiency'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-100-based_trained_5to50_Background_Efficiency.csv


In [9]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,559.22,349.76,314.03,312.99,312.95,294.00,0.07
DY,"14,495.72",671.44,671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,307.91,227.24,213.36,211.27,210.33,189.35,0.58
Triboson,53.00,4.32,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",9.31
Z+jets,"142,220.15","8,524.55","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",2.68
Total,"432,113.00","28,959.92","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",12.65


In [10]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to50_Background_EventCount_Weighted'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-100-based_trained_5to50_Background_EventCount_Weighted.csv


In [11]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,12311,381,381,160,159,159,158,3,0
Top,49818,4688,4688,3058,2348,2337,2336,2006,2
DY,205,13,13,11,11,11,11,9,0
Diboson,18067,1234,1234,915,860,852,848,767,2
Triboson,2257,170,170,116,106,105,104,92,0
W+jets,697657,48425,48425,37015,35472,35415,35377,29090,34
Z+jets,1281459,67427,67427,52627,50986,50934,50903,44480,21
Total,2061774,122338,122338,93902,89942,89813,89737,76447,59


In [12]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to50_Background_EventCount_Raw'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-100-based_trained_5to50_Background_EventCount_Raw.csv


In [13]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 12.65


In [14]:
pd.options.display.float_format = '{:,.2f}'.format

bkg_composition = pd.concat([pd.DataFrame(bkg_cts_df['BDT v2 Loose WP']).transpose(), pd.DataFrame(bkg_cts_raw_df['BDT v2 Loose WP']).transpose()], ignore_index=True)

bkg_composition = bkg_composition.rename(index={0: 'Weighted Counts', 1: 'Raw Counts'})
bkg_composition

,QCD,Top,DY,Diboson,Triboson,W+jets,Z+jets,Total
Weighted Counts,0.00,0.07,0.00,0.58,0.00,9.31,2.68,12.65
Raw Counts,0.00,2.00,0.00,2.00,0.00,34.00,21.00,59.00


## Signal

In [19]:
# Signal
sig_histo = util.load(f'{prefix}/signal{suffix}.coffea')[0]

In [20]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,84.0,8.0,100.0,80.0,88.0,0.1,sig_2018_Mchi-84p0_dMchi-8p0_ctau-100
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,88.0,16.0,100.0,80.0,96.0,0.2,sig_2018_Mchi-88p0_dMchi-16p0_ctau-100
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,99.0,18.0,100.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-100
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,94.5,9.0,100.0,90.0,99.0,0.1,sig_2018_Mchi-94p5_dMchi-9p0_ctau-100
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,77.0,14.0,100.0,70.0,84.0,0.2,sig_2018_Mchi-77p0_dMchi-14p0_ctau-100
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,60.0,20.0,100.0,50.0,70.0,0.4,sig_2018_Mchi-60p0_dMchi-20p0_ctau-100
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,73.5,7.0,100.0,70.0,77.0,0.1,sig_2018_Mchi-73p5_dMchi-7p0_ctau-100
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,63.0,6.0,100.0,60.0,66.0,0.1,sig_2018_Mchi-63p0_dMchi-6p0_ctau-100
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100


In [21]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'computing BDT score',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT v2 Loose WP']

### Efficiency

In [22]:
pd.options.display.float_format = '{:,.6f}'.format

In [23]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [24]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [25]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,0.036959,0.004105,0.004105,0.003461,0.003353,0.003349,0.003349,0.003245,0.000809
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.038143,0.004461,0.004461,0.003821,0.003680,0.003677,0.003677,0.003497,0.000921
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.042898,0.005171,0.005171,0.004361,0.004208,0.004206,0.004206,0.004008,0.000939
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,0.041559,0.004565,0.004565,0.003872,0.003775,0.003775,0.003775,0.003670,0.000896
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.032554,0.003600,0.003600,0.003015,0.002870,0.002870,0.002870,0.002746,0.000677
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.021028,0.002322,0.002322,0.001971,0.001921,0.001921,0.001921,0.001762,0.000521
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,0.032810,0.003467,0.003467,0.002935,0.002837,0.002837,0.002837,0.002743,0.000616
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,0.027190,0.002624,0.002624,0.002212,0.002100,0.002100,0.002100,0.002042,0.000490
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000054
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.000536


In [26]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-based_trained_5to50_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-based_trained_5to50_Signal_ctau-100_samples_Efficiency.csv


### Weighted Event count

In [27]:
pd.options.display.float_format = '{:,.2f}'.format

In [28]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [29]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [30]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,13.98,1.55,1.55,1.31,1.27,1.27,1.27,1.23,0.31
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.45,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.01
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.34,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,10.55,1.16,1.16,0.98,0.96,0.96,0.96,0.93,0.23
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.61,0.07,0.07,0.06,0.05,0.05,0.05,0.05,0.01
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,19.49,2.06,2.06,1.74,1.69,1.69,1.69,1.63,0.37
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,27.17,2.62,2.62,2.21,2.10,2.10,2.10,2.04,0.49
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.25,1.08,1.02,1.02,1.02,0.96,0.10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.15,0.13,0.12,0.12,0.12,0.12,0.03


In [31]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-based_trained_5to50_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-based_trained_5to50_Signal_ctau-100_samples_EventCount_Weighted.csv


In [32]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,13.98,1.55,1.55,1.31,1.27,1.27,1.27,1.23,0.31
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.45,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.01
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.34,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,10.55,1.16,1.16,0.98,0.96,0.96,0.96,0.93,0.23
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.61,0.07,0.07,0.06,0.05,0.05,0.05,0.05,0.01
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,19.49,2.06,2.06,1.74,1.69,1.69,1.69,1.63,0.37
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,27.17,2.62,2.62,2.21,2.10,2.10,2.10,2.04,0.49
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.25,1.08,1.02,1.02,1.02,0.96,0.10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.15,0.13,0.12,0.12,0.12,0.12,0.03


### Raw count

In [33]:
pd.options.display.float_format = '{:,.0f}'.format

In [34]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [35]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [36]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,8905,989,989,834,808,807,807,782,195
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,10603,1240,1240,1062,1023,1022,1022,972,256
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,23255,2803,2803,2364,2281,2280,2280,2173,509
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,25608,2813,2813,2386,2326,2326,2326,2261,552
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,7894,873,873,731,696,696,696,666,164
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,5452,602,602,511,498,498,498,457,135
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,9055,957,957,810,783,783,783,757,170
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,7990,771,771,650,617,617,617,600,144
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,190,165,155,155,155,147,16
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,754,643,617,617,617,590,167


In [37]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-based_trained_5to50_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-based_trained_5to50_Signal_ctau-100_samples_EventCount_Raw.csv


## S/sqrt(B)

In [38]:
nBkg

12.654182014616387

In [39]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [40]:
pd.options.display.float_format = '{:,.4f}'.format

In [44]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [45]:
# Missing rows
#df[ (df['m1'] == 60) & (df['delta'] == 0.2) ]

df.loc[len(df.index)] = [0, 60, 0.2, 100]

df = df.sort_values(by=['m1']) # sort by m1

In [46]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,2.3320,5.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.1050,5.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0716,10.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,2.0463,10.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,2.2315,20.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.0994,20.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,24.1107,30.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,338.3730,30.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.0707,40.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,1.9383,40.0000,0.1000,100.0000,12.6542


In [47]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,2.3320,12.6542,0.6556
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.1050,12.6542,0.0295
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0716,12.6542,0.0201
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,2.0463,12.6542,0.5752
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,2.2315,12.6542,0.6273
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0994,12.6542,0.0279
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,24.1107,12.6542,6.7779
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,338.3730,12.6542,95.1215
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0707,12.6542,0.0199
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,1.9383,12.6542,0.5449


In [48]:
outdir = './csv/SR_Yields/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to50_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_BDT_ctau-100-based_trained_5to50_S_over_B.csv


### ctau = 1mm signal points

In [49]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0,0.1,100.0,2.3320,12.6542,0.6556
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0,0.1,100.0,2.0463,12.6542,0.5752
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0,0.1,100.0,2.2315,12.6542,0.6273
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0,0.1,100.0,338.3730,12.6542,95.1215
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0,0.1,100.0,1.9383,12.6542,0.5449
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0,0.1,100.0,0.8567,12.6542,0.2408
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,60.0,0.1,100.0,0.4898,12.6542,0.1377
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,70.0,0.1,100.0,0.3661,12.6542,0.1029
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,80.0,0.1,100.0,0.3060,12.6542,0.0860
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,90.0,0.1,100.0,0.2274,12.6542,0.0639


In [50]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0,0.2,100.0,0.1050,12.6542,0.0295
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0,0.2,100.0,0.0716,12.6542,0.0201
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0,0.2,100.0,0.0994,12.6542,0.0279
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0,0.2,100.0,24.1107,12.6542,6.7779
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0,0.2,100.0,0.0707,12.6542,0.0199
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0,0.2,100.0,0.0327,12.6542,0.0092
22,60.0,0.2,100.0,0.0000,12.6542,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,70.0,0.2,100.0,0.0126,12.6542,0.0035
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,80.0,0.2,100.0,0.0109,12.6542,0.0031
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,90.0,0.2,100.0,0.0075,12.6542,0.0021
